# Find out merging units

In [2]:
import pandas as pd
import numpy as np

# Peaks ROI

In [3]:
project_name = 'SpatialCITE/humanskin/'
position = pd.read_csv(f'data/{project_name}/position_with_center.csv')

In [4]:
position

,index,barcode,row,col,imagerow,imagecol,is_center
0,0,45x44,45,44,2417,2359,0
1,1,20x14,20,14,1052,723,0
2,2,44x42,44,42,2363,2250,0
3,3,42x3,42,3,2253,123,1
4,4,11x40,11,40,560,2141,0
...,...,...,...,...,...,...,...
1613,1613,24x10,24,10,1270,505,0
1614,1614,15x29,15,29,778,1541,0
1615,1615,13x33,13,33,669,1759,0
1616,1616,32x48,32,48,1707,2577,0


In [7]:
adj_index = [[+1, +1], [+1, 0], [+1, -1],
             [+0, +1], [0, -1],
             [-1, +1], [-1, 0], [-1, -1],
            [+0, +0]]

In [9]:
dr = (1980-1926)/3
dc = (1050-995)/3
row = position['row'].tolist()
col = position['col'].tolist()
imagerow = position['imagerow'].tolist()
imagecol = position['imagecol'].tolist()

In [13]:
en_row = []
en_col = []
en_imagerow = []
en_imagecol = []

for i in range(len(row)):
    for j in range(9):
        en_row.append(row[i]*3+adj_index[j][0])
        en_col.append(col[i]*3+adj_index[j][1])
        en_imagerow.append(imagerow[i]+dr*adj_index[j][0])
        en_imagecol.append(imagecol[i]+dc*adj_index[j][1])
en_position =pd.DataFrame({
'row':en_row,
'col':en_col,
'imagerow':en_imagerow,
'imagecol':en_imagecol,
'is_center':([0]*8+[1])*len(row)
})
en_position

,row,col,imagerow,imagecol,is_center
0,136,133,2435.0,2377.333333,0
1,136,132,2435.0,2359.000000,0
2,136,131,2435.0,2340.666667,0
3,135,133,2417.0,2377.333333,0
4,135,131,2417.0,2340.666667,0
...,...,...,...,...,...
14557,60,110,1052.0,1958.666667,0
14558,59,112,1034.0,1995.333333,0
14559,59,111,1034.0,1977.000000,0
14560,59,110,1034.0,1958.666667,0


In [14]:
en_position.to_csv(f'data/{project_name}/x3/enhanced_position_with_center.csv')

# Image seg

In [15]:
import os
os.environ['OPENCV_IO_MAX_IMAGE_PIXELS'] = pow(2, 32).__str__()  # 2**32
import cv2
import json


In [16]:
image_ = f'data/{project_name}/skin.jpg'
he_image = cv2.imread(image_)
print(he_image.shape)

(2700, 2704, 3)


In [20]:
dr = (1980-1926)/3
dc = (1050-995)/3
ratio_base = np.sqrt(dr*dc)
ratio_base

18.16590212458495

In [30]:
def plot_dot(image, center, radius, color=0):
    if color == 1:
        color_value = (0, 0, 255)  # BGR
    elif color == 0:
        color_value = (255, 0, 0)  # BGR
    elif color == 2:
        color_value = (1, 1, 1)  # BGR
    else:
        color_value = (255, 255, 0)  # BGR
    dotted_img = cv2.circle(image, center, radius, color_value, -1)  # -1表示绘制实心园
    return dotted_img

def get_segment(image, center_y, center_x, ratio):
    image_dot = image[
                (center_y - ratio):(center_y + ratio),
                (center_x - ratio):(center_x + ratio),
                :]
    return image_dot

out_dim = (299, 299)
dot_image = he_image.copy()
path = f'data/{project_name}/x3'
for scale in range(2, 3):
    ratio = int(ratio_base * scale)
    count = 0
    for i in range(len(en_position['is_center'])):
        flag_center = en_position['is_center'][i]
        center = (int(en_imagerow[i]), int(en_imagecol[i]))
        dot_image = plot_dot(dot_image, center, 4, flag_center)
        seg_image = get_segment(he_image, int(en_imagecol[i]), int(en_imagerow[i]), ratio)
        resize_image = cv2.resize(seg_image, out_dim, interpolation=cv2.INTER_CUBIC)
        count += 1
        if count % 100 == 0:
            print(count)
        image_fold = path + f'\\image_x{scale}\\'
        if not os.path.exists(image_fold):
            os.makedirs(image_fold)
        cv2.imwrite(path + f'\\image_x{scale}\\' + 'Img_' + str(count) + '.jpg', resize_image)
cv2.imwrite(f'{path}/skin_dotted.tif', dot_image)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500


True